In [4]:
import sys
import pprint
from pathlib import Path
project_root = str(Path.cwd().parent)
if project_root not in sys.path:
 sys.path.append(project_root)
 
from pipeline.utils.general import load_json_data
from pipeline.utils.path_manager import PathManager
from evaluation.tail_posture_analysis.analysis import TailPostureAnalyzer
from evaluation.early_warning_analysis.early_warning_analysis import EarlyWarningAnalyzer 

path_manager = PathManager()

In [6]:
config_tail_posture_evaluation = load_json_data('config_tail_posture.json')

pprint.pprint(config_tail_posture_evaluation, compact=True)

{'analysis_window_days': [1, 3, 5, 7],
 'comparison_metrics': ['value_at_removal', '1d_window_avg', '3d_window_avg',
                        '5d_window_avg', '7d_window_avg', '1d_window_slope',
                        '3d_window_slope', '5d_window_slope', '7d_window_slope',
                        'abs_change_1d', 'abs_change_3d', 'abs_change_5d',
                        'abs_change_7d'],
 'comparison_stats_filename': 'outbreak_vs_control_comparison.csv',
 'component_analysis_window_days': 10,
 'component_timepoint_days': [0, 1, 3, 5, 7],
 'confidence_level': 0.95,
 'control_components_filename': 'control_posture_components.csv',
 'control_date_margin': 5,
 'control_samples_per_pen': 5,
 'control_stats_filename': 'control_statistics.csv',
 'days_before_list': [1, 3, 5, 7],
 'figure_dpi': 600,
 'interpolate_resampled_data': False,
 'max_allowed_consecutive_missing_days': 3,
 'max_allowed_missing_days_pct': 50.0,
 'min_control_analysis_dates': 2,
 'min_control_dates_threshold': 8,
 'min_

In [7]:
# Initialize the analyzer with your config
analyzer = TailPostureAnalyzer(config_tail_posture_evaluation)

# Load and preprocess data
analyzer.load_data()

# Preprocess the data
analyzer.preprocess_monitoring_results()

# Run standard analyses
analyzer.analyze_pre_outbreak_statistics()
analyzer.analyze_control_pen_statistics()
analyzer.compare_outbreak_vs_control_statistics()
analyzer.analyze_individual_outbreak_variation()
analyzer.analyze_posture_components()

2025-05-20 07:20:32,581 - INFO - Saved configuration to results_tail_posture_evaluation/analysis_config.txt
2025-05-20 07:20:32,582 - INFO - Loading monitoring pipeline data...
2025-05-20 07:20:39,044 - WARNING - Missing 1 day(s) for Kamera5/221004_221104: [datetime.date(2022, 11, 3)]
2025-05-20 07:20:41,038 - WARNING - Missing 17 day(s) for Kamera5/220510_220529: [datetime.date(2022, 5, 10), datetime.date(2022, 5, 12), datetime.date(2022, 5, 14), datetime.date(2022, 5, 15), datetime.date(2022, 5, 16), datetime.date(2022, 5, 18), datetime.date(2022, 5, 19), datetime.date(2022, 5, 20), datetime.date(2022, 5, 21), datetime.date(2022, 5, 22), datetime.date(2022, 5, 23), datetime.date(2022, 5, 24), datetime.date(2022, 5, 25), datetime.date(2022, 5, 26), datetime.date(2022, 5, 27), datetime.date(2022, 5, 28), datetime.date(2022, 5, 29)]
2025-05-20 07:20:41,049 - WARNING - Dataframe became empty after time filtering: Kamera5_2022_05_11_08_18_23-09_18_23.csv. Skipping.
2025-05-20 07:20:41,060

{'outbreak_components':        pen       datespan  days_before_removal       date  upright_tails  \
 0    Pen 1  220228_220328                    0 2022-03-23       0.387648   
 1    Pen 1  220228_220328                    1 2022-03-22       0.528087   
 2    Pen 1  220228_220328                    2 2022-03-21       0.682334   
 3    Pen 1  220228_220328                    3 2022-03-20       0.767497   
 4    Pen 1  220228_220328                    4 2022-03-19       0.855052   
 ..     ...            ...                  ...        ...            ...   
 215  Pen 6  230515_230622                    6 2023-05-26       0.802259   
 216  Pen 6  230515_230622                    7 2023-05-25       0.791398   
 217  Pen 6  230515_230622                    8 2023-05-24       0.787130   
 218  Pen 6  230515_230622                    9 2023-05-23       0.783088   
 219  Pen 6  230515_230622                   10 2023-05-22       0.783680   
 
      hanging_tails  posture_diff     group  
 0   

In [8]:
# early warning analysis
config_early_warning_analysis = load_json_data('config_early_warning.json')

pprint.pprint(config_early_warning_analysis, compact=True)

{'default_threshold_sets': [{'thresholds': {'alert': {'posture_diff': 0.4},
                                            'attention': {'posture_diff': 0.6},
                                            'critical': {'posture_diff': 0.2}}}],
 'details_filename': 'early_warning_pen_details.csv',
 'max_days_before': 7,
 'optimization_filename': 'threshold_optimization_results.csv',
 'output_dir': 'results_early_warning_evaluation',
 'random_seed': 42,
 'summary_filename': 'early_warning_evaluation.csv',
 'thresholds': {'alert': {'posture_diff': 0.4},
                'attention': {'posture_diff': 0.5},
                'critical': {'posture_diff': 0.25}},
 'use_interpolated_data': True}


In [9]:
evaluator = EarlyWarningAnalyzer(analyzer, config=config_early_warning_analysis)
warning_evaluation = evaluator.evaluate_thresholds()

2025-05-20 07:22:24,975 - INFO - EarlyWarningAnalyzer initialized with configuration:
2025-05-20 07:22:24,976 - INFO -   random_seed: 42
2025-05-20 07:22:24,976 - INFO -   use_interpolated_data: True
2025-05-20 07:22:24,977 - INFO -   max_days_before: 7
2025-05-20 07:22:24,977 - INFO -   thresholds: {'attention': {'posture_diff': 0.5}, 'alert': {'posture_diff': 0.4}, 'critical': {'posture_diff': 0.25}}
2025-05-20 07:22:24,978 - INFO -   output_dir: results_early_warning_evaluation
2025-05-20 07:22:24,978 - INFO -   summary_filename: early_warning_evaluation.csv
2025-05-20 07:22:24,979 - INFO -   details_filename: early_warning_pen_details.csv
2025-05-20 07:22:24,979 - INFO -   optimization_filename: threshold_optimization_results.csv
2025-05-20 07:22:24,979 - INFO - Evaluating early warning thresholds...
2025-05-20 07:22:24,980 - INFO - Including the following threshold levels in evaluation: ['attention', 'alert', 'critical']
2025-05-20 07:22:24,981 - INFO - Using interpolated_data for

In [10]:
# Print key results
print("\nEarly Warning System Evaluation Results:")
print("----------------------------------------")
print(f"Attention Level - Sensitivity: {warning_evaluation['attention_level']['metrics']['sensitivity']:.2f}")
print(f"Attention Level - Specificity: {warning_evaluation['attention_level']['metrics']['specificity']:.2f}")
print(f"Attention Level - Average Warning Time: {warning_evaluation['attention_level']['avg_days_before_outbreak']:.1f} days")

print("----------------------------------------")
print(f"Alert Level - Sensitivity: {warning_evaluation['alert_level']['metrics']['sensitivity']:.2f}")
print(f"Alert Level - Specificity: {warning_evaluation['alert_level']['metrics']['specificity']:.2f}")
print(f"Alert Level - Average Warning Time: {warning_evaluation['alert_level']['avg_days_before_outbreak']:.1f} days")

print("----------------------------------------")
print(f"Critical Level - Sensitivity: {warning_evaluation['critical_level']['metrics']['sensitivity']:.2f}")
print(f"Critical Level - Specificity: {warning_evaluation['critical_level']['metrics']['specificity']:.2f}")
print(f"Critical Level - Average Warning Time: {warning_evaluation['critical_level']['avg_days_before_outbreak']:.1f} days")


Early Warning System Evaluation Results:
----------------------------------------
Attention Level - Sensitivity: 0.85
Attention Level - Specificity: 0.55
Attention Level - Average Warning Time: 4.1 days
----------------------------------------
Alert Level - Sensitivity: 0.85
Alert Level - Specificity: 0.75
Alert Level - Average Warning Time: 3.7 days
----------------------------------------
Critical Level - Sensitivity: 0.80
Critical Level - Specificity: 0.90
Critical Level - Average Warning Time: 3.2 days


In [11]:
# Evaluate all levels (default behavior)
all_results = evaluator.evaluate_thresholds()

# Evaluate only attention level
attention_results = evaluator.evaluate_thresholds(include_levels=['attention'])

# Evaluate alert and critical levels together
alert_critical_results = evaluator.evaluate_thresholds(include_levels=['alert', 'critical'])

# Evaluate each level individually for comparison
attention_only = evaluator.evaluate_thresholds(include_levels=['attention'])
alert_only = evaluator.evaluate_thresholds(include_levels=['alert'])
critical_only = evaluator.evaluate_thresholds(include_levels=['critical'])

2025-05-20 07:23:03,536 - INFO - Evaluating early warning thresholds...
2025-05-20 07:23:03,537 - INFO - Including the following threshold levels in evaluation: ['attention', 'alert', 'critical']
2025-05-20 07:23:03,538 - INFO - Using interpolated_data for threshold evaluation
2025-05-20 07:23:04,769 - INFO - Saved early warning evaluation to results_early_warning_evaluation/early_warning_evaluation.csv
2025-05-20 07:23:04,771 - INFO - 
Early Warning System Evaluation:
2025-05-20 07:23:04,771 - INFO - Levels included: ['attention', 'alert', 'critical']
2025-05-20 07:23:04,772 - INFO - Total pens analyzed: 40 (20 outbreak, 20 control)
2025-05-20 07:23:04,772 - INFO - 
Attention Level Metrics:
2025-05-20 07:23:04,773 - INFO - Sensitivity: 0.85
2025-05-20 07:23:04,773 - INFO - Specificity: 0.55
2025-05-20 07:23:04,773 - INFO - Average warning time: 98.9 hours (4.1 days)
2025-05-20 07:23:04,774 - INFO - 
Alert Level Metrics:
2025-05-20 07:23:04,774 - INFO - Sensitivity: 0.85
2025-05-20 07:

In [12]:
evaluator.print_threshold_analysis_results(all_results)


=== EARLY WARNING SYSTEM EVALUATION SUMMARY ===
Total pens analyzed: 40 (20 outbreak, 20 control)
Threshold levels included: attention, alert, critical

Level      | Sens.      | Spec.      | Prec.      | F1         | Avg Warning     | Med Warning    
------------------------------------------------------------------------------------------
Attention  | 0.85       | 0.55       | 0.65       | 0.74       | 4d 2.9h         | 3d 8.0h        
Alert      | 0.85       | 0.75       | 0.77       | 0.81       | 3d 16.1h        | 3d 1.0h        
Critical   | 0.80       | 0.90       | 0.89       | 0.84       | 3d 5.1h         | 2d 14.5h       

=== CONFUSION MATRIX INFORMATION ===

Attention Level:
  True Positives:  17 (Outbreak pen correctly flagged)
  False Positives: 9 (Control pen incorrectly flagged)
  True Negatives:  11 (Control pen correctly not flagged)
  False Negatives: 3 (Outbreak pen incorrectly not flagged)

Alert Level:
  True Positives:  17 (Outbreak pen correctly flagged)
  Fals

### Visualizations

In [ ]:
from evaluation.early_warning_analysis.early_warning_visualization import EarlyWarningVisualizer

vis = EarlyWarningVisualizer(evaluator)

vis.visualize_all_pens()

2025-05-20 07:23:55,930 - INFO - Setting plot style: Font=serif, BaseSize=11pt, Format=png, DPI=600


INFO: EarlyWarningVisualizer initialized. Visualizations will be saved to results_early_warning_evaluation/visualizations
INFO: Creating visualization for pen Pen 3 / 211222_220119
INFO: Visualization saved to results_early_warning_evaluation/visualizations/Pen_3_211222_220119_control_early_warning.png
INFO: Creating visualization for pen Pen 2 / 211227_220119
INFO: Visualization saved to results_early_warning_evaluation/visualizations/Pen_2_211227_220119_control_early_warning.png
INFO: Creating visualization for pen Pen 4 / 221226_230126
INFO: Visualization saved to results_early_warning_evaluation/visualizations/Pen_4_221226_230126_tail_biting_early_warning.png
INFO: Creating visualization for pen Pen 5 / 211231_220119
INFO: Visualization saved to results_early_warning_evaluation/visualizations/Pen_5_211231_220119_control_early_warning.png
INFO: Creating visualization for pen Pen 1 / 230515_230620
INFO: Visualization saved to results_early_warning_evaluation/visualizations/Pen_1_2305

[<Figure size 864x576 with 1 Axes>,
 <Figure size 864x576 with 1 Axes>,
 <Figure size 864x576 with 1 Axes>,
 <Figure size 864x576 with 1 Axes>,
 <Figure size 864x576 with 1 Axes>,
 <Figure size 864x576 with 1 Axes>,
 <Figure size 864x576 with 1 Axes>,
 <Figure size 864x576 with 1 Axes>,
 <Figure size 864x576 with 1 Axes>,
 <Figure size 864x576 with 1 Axes>,
 <Figure size 864x576 with 1 Axes>,
 <Figure size 864x576 with 1 Axes>,
 <Figure size 864x576 with 1 Axes>,
 <Figure size 864x576 with 1 Axes>,
 <Figure size 864x576 with 1 Axes>,
 <Figure size 864x576 with 1 Axes>,
 <Figure size 864x576 with 1 Axes>,
 <Figure size 864x576 with 1 Axes>,
 <Figure size 864x576 with 1 Axes>,
 <Figure size 864x576 with 1 Axes>,
 <Figure size 864x576 with 1 Axes>,
 <Figure size 864x576 with 1 Axes>,
 <Figure size 864x576 with 1 Axes>,
 <Figure size 864x576 with 1 Axes>,
 <Figure size 864x576 with 1 Axes>,
 <Figure size 864x576 with 1 Axes>,
 <Figure size 864x576 with 1 Axes>,
 <Figure size 864x576 with 1

: 